# L5: Content Creation at Scale

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Creating Structured Output

In [2]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

## Loading Tasks and Agents YAML files

In [3]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Importing CrewAI Tools

In [4]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

## Setup Multi LLM models

In [5]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
groq_llm = "groq/llama-3.1-70b-versatile"

## Creating Crew, Agents, and Tasks

In [6]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=groq_llm,
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
    llm=groq_llm,
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)

## Kicking off the Crew

In [7]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'Inflation in the US and the impact on the stock market in 2024'
})

# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on Inflation in the US and the impact on the stock market in 2024. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to Inflation in the US and the impact on the stock market in 2024.



# Agent: Lead Market Analyst
## Thought: Thought: To gather the most relevant and up-to-date information on Inflation in the US and its impact on the stock market in 2024, I should search the internet for recent news articles and updates from reputable financial sources.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"US inflation news 2024 and stock market impact\"}"
##



# Agent: Lead Market Analyst
## Final Answer: 
Recent news articles and updates from reputable financial sources indicate that the relationship between inflation and the stock market in 2024 is complex and influenced by various factors. The current inflation rate in the US is 2.4% for the 12 months ending September 2024, which is lower than the previous rate increase of 2.5%.

According to the Consumer Price Index (CPI) data released by the Labor Department on October 10, inflation cooled to 2.4% in September 2024. This decrease in inflation has led to a rally in the stock market, with major indices such as the Dow Jones Industrial Average and S&P 500 repeatedly hitting all-time highs.

However, some experts warn that rising interest rates could pose a test for the stock rally. The Federal Reserve has been increasing interest rates to combat inflation, which could lead to a decrease in consumer spending and a subsequent decline in the stock market.

Other factors that could impact th



# Agent: Chief Data Strategist
## Thought: Thought: Based on the information gathered from the search results, it appears that the Federal Reserve has been adjusting its monetary policy in response to changes in inflation and the economy. The Fed has cut interest rates twice in 2024, with the latest cut being 0.25 percentage points in November. This move is expected to boost the economy and stock market.
However, I need to gather more information on the potential risks and challenges associated with the current economic trends. I also want to know more about the relationship between inflation, interest rates, and the stock market from the perspective of other financial experts and organizations.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Financial expert opinions on inflation rate, interest rates impact on stock market 2024\"}"
## Tool Output: 

Search results: Title: How Do Changing Interest Rates Affect the Stock Market? | U.S. Bank
Link: https://www.u

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  8.54it/s]




# Agent: Chief Data Strategist
## Thought: No worries, let's try again.
Thought: I want to search for more information on the relationship between inflation, interest rates, and the stock market from the perspective of experts at the Federal Reserve.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"Federal Reserve experts opinions on inflation rate, interest rates impact on stock market 2024\", \"website\": \"https://www.federalreserve.gov/\"}"
## Tool Output: 
Relevant Content:
outlook. In this scenario, continued geopolitical conflicts and trade actions cause inflation to stay higher for longer, driving the Federal Reserve to hike rates.1 Scenarios Baseline (70%): Real GDP growth slowed overall in the first quarter of 2024, coming in very close to our previous forecast’s expectations.2 We expect GDP growth will continue to moderate through the second half of this year and the start of the next, but the story is still positive overall. Consumer spend



# Agent: Creative Content Director
## Thought: I need to gather the latest insights and data regarding inflation in the US and its impact on the stock market in 2024 to create high-quality, engaging content that includes blog posts and social media updates.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"US inflation impact stock market 2024 analysis\"}"
## Tool Output: 

Search results: Title: Inflation Artificially Pumps Up the Stock Market
Link: https://www.heritage.org/budget-and-spending/commentary/inflation-artificially-pumps-the-stock-market
Snippet: While the nominal stock market gains since 2021 show an increase of 39%, this growth shrinks to just 15% when adjusted for inflation.
---
Title: US Economic Forecast Q3 2024 | Deloitte Insights
Link: https://www2.deloitte.com/us/en/insights/economy/us-economic-forecast/united-states-outlook-analysis.html
Snippet: The Q3 2024 forecast indicates how robust consumer spending, high business investment, and low

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00, 32.40it/s]




# Agent: Creative Content Director
## Thought: Thought: I need to gather more specific insights directly from a respected financial source regarding inflation and the stock market impact in 2024.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"How Inflation Affects The Stock Market\", \"website\": \"https://www.bankrate.com\"}"
## Tool Output: 
Relevant Content:
Investopedia What the Dockworkers' Strike Could Cost the U.S. Economy U.S. dockworkers in ports from New England to Texas have started to strike, which could cost the U.S. economy up to $4.5 billion daily and reignite inflation. By Elizabeth Guevara 9 minutes ago Stocks Slide as Tech Tumbles; Oil Up on Middle East Tension Stocks retreated from record highs as big tech stumbled, while oil prices jumped on concerns about conflict in the Middle East. By Stephen Wisnefski 20 minutes ago Related What's in Store for Stocks in Fourth Quarter? Five Stocks to Watch in October other top stories Nike Dr



# Agent: Chief Content Officer
## Final Answer: 
```markdown
# Understanding US Inflation and Its Complex Relationship with the Stock Market in 2024

## Introduction

The economic landscape in 2024 has been significantly shaped by inflation dynamics in the United States. With a current inflation rate of **2.4%** for the 12 months ending September 2024, the interplay between inflation and stock market performance is intricate. This blog post explores the trends and factors contributing to the current financial environment, as well as insights and recommendations for investors looking to navigate these waters in 2024.

## The Current State of Inflation

As reported by the Labor Department, inflation has cooled to **2.4%** in September, offering a glimpse of relief for consumers and investors alike. This decrease, compared to last year's **2.5%**, has sparked excitement in the stock market, leading to significant rallies. Major indices such as the **Dow Jones Industrial Average** and th

## Social Content

In [8]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

Twitter
📈 US inflation hits 2.4% in September 2024! How
does this impact the stock market? Let’s explore
the complexities and opportunities for investors.
#Inflation #StockMarket #Investing
--------------------------------------------------
LinkedIn
The relationship between inflation and stock
market performance is intricate. With inflation
cooling to 2.4% and the Fed adjusting rates,
investors must be prepared for potential
challenges ahead in 2024. Join me as we uncover
the key trends. #Finance #MarketInsights
--------------------------------------------------
Instagram
🚨 Inflation Update: The US inflation rate has
dropped to 2.4%! 📊 How will this affect your
investments in 2024? Stay tuned for insights and
strategies! #InvestSmart #EconomicTrends
--------------------------------------------------


## Blog Post

In [9]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))

# Understanding US Inflation and Its Complex Relationship with the Stock Market in 2024

## Introduction

The economic landscape in 2024 has been significantly shaped by inflation dynamics in the United States. With a current inflation rate of **2.4%** for the 12 months ending September 2024, the interplay between inflation and stock market performance is intricate. This blog post explores the trends and factors contributing to the current financial environment, as well as insights and recommendations for investors looking to navigate these waters in 2024.

## The Current State of Inflation

As reported by the Labor Department, inflation has cooled to **2.4%** in September, offering a glimpse of relief for consumers and investors alike. This decrease, compared to last year's **2.5%**, has sparked excitement in the stock market, leading to significant rallies. Major indices such as the **Dow Jones Industrial Average** and the **S&P 500** have repeatedly reached all-time highs, driven by optimistic market sentiment and robust consumer spending.

![Inflation Trends](https://via.placeholder.com/300)

The positive market reaction is indicative of the resilience shown by consumers despite the previous economic pressures. Investors are particularly responding to the modest inflation as a potential sign that the economic recovery is gaining a foothold. Analysts suggest that this could be a stable period where growth is coupled with controlled inflation, setting the stage for sustained market performance.

## Impact of Rising Interest Rates

Despite the positive market reaction to lower inflation, experts caution about the potential challenges that rising interest rates may present. The **Federal Reserve** has been adjusting monetary policies in response to inflation and economic conditions, including two rate cuts in 2024. While lower interest rates tend to boost stock market performance by encouraging consumer spending, ongoing geopolitical tensions and unexpected economic events could trigger rate hikes, stifling market growth and consumer confidence.

Furthermore, a sudden shift in monetary policy can lead to volatility in the stock market as investors recalibrate their strategies based on new economic conditions. Maintaining a close watch on the Federal Reserve's announcements and policy decisions will be vital for market participants aiming to navigate these turbulent waters.

## Key Factors Influencing the Stock Market Outlook

1. **Consumer Spending**: Consumer spending remains robust, contributing to economic growth. Predictions indicate a **2.3%** increase in spending for 2024, which is vital for stock market optimism.
  
2. **Presidential Election Outcome**: Political dynamics also play a crucial role. Experts predict that a *Republican victory* in the upcoming presidential election may lead to favorable market conditions.

3. **Geopolitical Tensions**: Continued global conflicts could keep inflation elevated for longer, prompting the Federal Reserve to exercise caution when adjusting interest rates.

4. **Investment Shifts**: Economic performance indicators suggest that sectors like technology and manufacturing are poised for expansion, influenced by the **Inflation Reduction Act** and advancements in technology.

Each of these factors will be critical for investors to consider as they develop strategies for the upcoming year. Being attuned to changes in consumer behavior, market sentiment, and political shakeups will contribute to more informed decision-making.

## Conclusion

Navigating the complex relationship between inflation and stock market dynamics will require vigilance and strategic planning from investors in 2024. With an inflation forecast anticipated to ease to **2.7%** by the end of the year, observing Federal Reserve actions and consumer behavior will be essential. As investors, staying informed on these trends and preparing for potential economic shifts can prove invaluable for making sound investment decisions.

---

## Social Media Updates

- **Tweet**: "📈 US inflation hits 2.4% in September 2024! How does this impact the stock market? Let’s explore the complexities and opportunities for investors. #Inflation #StockMarket #Investing"

- **LinkedIn Post**: "The relationship between inflation and stock market performance is intricate. With inflation cooling to 2.4% and the Fed adjusting rates, investors must be prepared for potential challenges ahead in 2024. Join me as we uncover the key trends. #Finance #MarketInsights"

- **Instagram Story**: "🚨 Inflation Update: The US inflation rate has dropped to 2.4%! 📊 How will this affect your investments in 2024? Stay tuned for insights and strategies! #InvestSmart #EconomicTrends"

Through multimedia elements, infographics, and engaging storytelling, this content encapsulates the current economic sentiments and provides actionable insights for investors looking to make informed decisions in a potentially volatile market landscape for 2024.